## Build an ML Pipeline for Airfoil noise prediction


## Scenario


In this project we will use the modified version of the NASA Airfoil Self Noise dataset. We will clean this dataset, by dropping the duplicate rows, and removing the rows with null values. We will create an ML pipe line to create a model that will predict the SoundLevel based on all the other columns. We will evaluate the model and towards the end you will persist the model.


## Objectives

- Part 1 Perform ETL activity
  - Load a csv dataset
  - Remove duplicates if any
  - Drop rows with null values if any
  - Make transformations
  - Store the cleaned data in parquet format
- Part 2 Create a  Machine Learning Pipeline
  - Create a machine learning pipeline for prediction
- Part 3 Evaluate the Model
  - Evaluate the model using relevant metrics
- Part 4 Persist the Model
  - Save the model for future production use
  - Load and verify the stored model


## Datasets

We will be using dataset(s):

 - The original dataset can be found here NASA airfoil self noise dataset. https://archive.ics.uci.edu/dataset/291/airfoil+self+noise

 - This dataset is licensed under a Creative Commons Attribution 4.0 International (CC BY 4.0) license.


Diagram of an airfoil. - For informational purpose


![Airfoil with flow](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/images/Airfoil_with_flow.png)


Diagram showing the Angle of attack. - For informational purpose


![Airfoil angle of attack](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/images/Airfoil_angle_of_attack.jpg)


In [1]:
!pip install pyspark==3.1.2 -q
!pip install findspark -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.4/212.4 MB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 21.2 MB/s eta 0:00:00


In [2]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

# FindSpark simplifies the process of using Apache Spark with Python

import findspark
findspark.init()

## Part 1 - Perform ETL activity


In [3]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.pipeline import PipelineModel
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import StandardScaler

In [4]:
#Create SparkSession

spark = SparkSession.builder.appName("Practice Project").getOrCreate()

In [5]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/datasets/NASA_airfoil_noise_raw.csv

--2023-11-25 18:25:24--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/datasets/NASA_airfoil_noise_raw.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60682 (59K) [text/csv]
Saving to: ‘NASA_airfoil_noise_raw.csv’

NASA_airfoil_noise_ 100%[===================>]  59.26K  --.-KB/s    in 0.007s  

2023-11-25 18:25:24 (7.75 MB/s) - ‘NASA_airfoil_noise_raw.csv’ saved [60682/60682]



Load the dataset into the spark dataframe


In [6]:
# Load the dataset that you have downloaded in the previous task

df = spark.read.csv("NASA_airfoil_noise_raw.csv", header=True, inferSchema=True)

In [7]:
df.show(5)

+---------+-------------+-----------+------------------+-----------------------+----------+
|Frequency|AngleOfAttack|ChordLength|FreeStreamVelocity|SuctionSideDisplacement|SoundLevel|
+---------+-------------+-----------+------------------+-----------------------+----------+
|      800|          0.0|     0.3048|              71.3|             0.00266337|   126.201|
|     1000|          0.0|     0.3048|              71.3|             0.00266337|   125.201|
|     1250|          0.0|     0.3048|              71.3|             0.00266337|   125.951|
|     1600|          0.0|     0.3048|              71.3|             0.00266337|   127.591|
|     2000|          0.0|     0.3048|              71.3|             0.00266337|   127.461|
+---------+-------------+-----------+------------------+-----------------------+----------+
only showing top 5 rows



In [8]:
rowcount1 = df.count()
print(rowcount1)

1522


In [9]:
df = df.dropDuplicates()

In [10]:
rowcount2 = df.count()
print(rowcount2)

1503


In [11]:
df = df.dropna()

In [12]:
rowcount3 = df.count()
print(rowcount3)

1499


In [13]:
#Rename the column "SoundLevel" to "SoundLevelDecibels"Drop

df = df.withColumnRenamed("SoundLevel","SoundLevelDecibels")

In [14]:
df.write.mode("overwrite").parquet("NASA_airfoil_noise_cleaned.parquet")

In [15]:
print("Part 1 - Evaluation")

print("Total rows = ", rowcount1)
print("Total rows after dropping duplicate rows = ", rowcount2)
print("Total rows after dropping duplicate rows and rows with null values = ", rowcount3)
print("New column name = ", df.columns[-1])

import os

print("NASA_airfoil_noise_cleaned.parquet exists :", os.path.isdir("NASA_airfoil_noise_cleaned.parquet"))

Part 1 - Evaluation
Total rows =  1522
Total rows after dropping duplicate rows =  1503
Total rows after dropping duplicate rows and rows with null values =  1499
New column name =  SoundLevelDecibels
NASA_airfoil_noise_cleaned.parquet exists : True


## Part - 2 Create a  Machine Learning Pipeline


In [16]:
df = spark.read.parquet("NASA_airfoil_noise_cleaned.parquet")

In [17]:
rowcount4 = df.count()
print(rowcount4)

1499


In [18]:
df.show(5)

+---------+-------------+-----------+------------------+-----------------------+------------------+
|Frequency|AngleOfAttack|ChordLength|FreeStreamVelocity|SuctionSideDisplacement|SoundLevelDecibels|
+---------+-------------+-----------+------------------+-----------------------+------------------+
|      630|          0.0|     0.3048|              31.7|             0.00331266|           129.095|
|     4000|          0.0|     0.3048|              31.7|             0.00331266|           118.145|
|     4000|          1.5|     0.3048|              39.6|             0.00392107|           117.741|
|      800|          4.0|     0.3048|              71.3|             0.00497773|           131.755|
|     1250|          0.0|     0.2286|              31.7|              0.0027238|           128.805|
+---------+-------------+-----------+------------------+-----------------------+------------------+
only showing top 5 rows



Stage 1 - Assemble the input columns into a single column "features". Use all the columns except SoundLevelDecibels as input features.


In [19]:
assembler = VectorAssembler(inputCols=['Frequency','AngleOfAttack','ChordLength','FreeStreamVelocity','SuctionSideDisplacement'], outputCol="features")

Stage 2 - Scale the "features" using standard scaler and store in "scaledFeatures" column


In [20]:
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")

Stage 3 - Create a LinearRegression stage to predict "SoundLevelDecibels"

**Note: We need to use the scaledfeatures retreived in the previous step.**


In [21]:
lr = LinearRegression(featuresCol="scaledFeatures", labelCol="SoundLevelDecibels")

Build a pipeline using the above three stages


In [22]:
pipeline = Pipeline(stages=[assembler, scaler, lr])

In [23]:
# Split the data into training and testing sets with 70:30 split.
# set the value of seed to 42
(trainingData, testingData) = df.randomSplit([0.7, 0.3], seed=42)

In [24]:
# Fit the pipeline using the training data
pipelineModel = pipeline.fit(trainingData)

In [25]:
print("Part 2 - Evaluation")
print("Total rows = ", rowcount4)
ps = [str(x).split("_")[0] for x in pipeline.getStages()]

print("Pipeline Stage 1 = ", ps[0])
print("Pipeline Stage 2 = ", ps[1])
print("Pipeline Stage 3 = ", ps[2])

print("Label column = ", lr.getLabelCol())

Part 2 - Evaluation
Total rows =  1499
Pipeline Stage 1 =  VectorAssembler
Pipeline Stage 2 =  StandardScaler
Pipeline Stage 3 =  LinearRegression
Label column =  SoundLevelDecibels


In [26]:
# Make predictions on testing data

predictions = pipelineModel.transform(testingData)

In [27]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="SoundLevelDecibels", metricName="mse")
mse = evaluator.evaluate(predictions)
print(mse)

24.354422246772728


In [28]:
evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="SoundLevelDecibels", metricName="mae")
mae = evaluator.evaluate(predictions)
print(mae)

3.7287658285335303


In [29]:
evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="SoundLevelDecibels", metricName="r2")
r2 = evaluator.evaluate(predictions)
print(r2)

0.5053765847518492


In [30]:
print("Part 3 - Evaluation")

print("Mean Squared Error = ", round(mse,2))
print("Mean Absolute Error = ", round(mae,2))
print("R Squared = ", round(r2,2))

lrModel = pipelineModel.stages[-1]

print("Intercept = ", round(lrModel.intercept,2))

Part 3 - Evaluation
Mean Squared Error =  24.35
Mean Absolute Error =  3.73
R Squared =  0.51
Intercept =  132.61


## Part 4 - Persist the Model


In [31]:
# Save the pipeline model as "Final_Project"

pipelineModel.write().overwrite().save("Final_Project")

In [32]:
# Load the pipeline model you have created in the previous step
loadedPipelineModel = PipelineModel.load("Final_Project")

In [33]:
# Use the loaded pipeline model and make predictions using testingData
predictions = loadedPipelineModel.transform(testingData)

In [34]:
#show top 5 rows from the predections dataframe. Display only the label column and predictions
predictions.select("SoundLevelDecibels","prediction").show()

+------------------+------------------+
|SoundLevelDecibels|        prediction|
+------------------+------------------+
|           128.545|121.25440562914048|
|           130.898|122.70281104682431|
|           109.951|127.80380966422578|
|           112.506|129.44660894124058|
|           130.089|122.22655518784543|
|           139.918|126.62151483038025|
|           134.533| 130.6845914583634|
|           123.312|123.11545113389359|
|           125.741|126.95725865339958|
|           139.808|126.54113252730056|
|           128.595|123.06839887156663|
|           128.484|121.03743090017933|
|           123.537|128.39739892038955|
|           122.938|131.67804533414386|
|           124.493|123.93486336148476|
|           122.754|117.89354237100211|
|           115.846|129.30439409733037|
|           131.221|124.74693531399491|
|           135.924|124.74907154871491|
|           120.076| 129.3641483351852|
+------------------+------------------+
only showing top 20 rows



In [35]:
print("Part 4 - Evaluation")

loadedmodel = loadedPipelineModel.stages[-1]
totalstages = len(loadedPipelineModel.stages)
inputcolumns = loadedPipelineModel.stages[0].getInputCols()

print("Number of stages in the pipeline = ", totalstages)
for i,j in zip(inputcolumns, loadedmodel.coefficients):
    print(f"Coefficient for {i} is {round(j,4)}")

Part 4 - Evaluation
Number of stages in the pipeline =  3
Coefficient for Frequency is -3.8894
Coefficient for AngleOfAttack is -2.1888
Coefficient for ChordLength is -3.4035
Coefficient for FreeStreamVelocity is 1.5495
Coefficient for SuctionSideDisplacement is -2.1905


### Stop Spark Session


In [36]:
spark.stop()